In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import math as m
from scipy.stats import t
from scipy.optimize import minimize_scalar
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import copy
%matplotlib inline

In [ ]:
def Anomaly_exp(df_input, porosity, TC, draw, sample, confidence_interval):  
    
    '''Функция предназначена для отсеивания аномальных образцов при экспоненциальной зависимости
       Аргументы:
       df_input — входная таблица, тип — Pandas Dataframe;
       porosity — заголовок столбца с пористостью в df_input;
       TC — заголовок столбца с теплопроводностью (как правило, лямбда паралл.) в df_input;
       draw — если значение True — визуализация исходных данных и выделение аномальных образцов;
       sample — заголовок столбца с названиями/номерами образцов;
       confidence_interval — доверительный интервал при отборе аномальных образцов. Как правило, равен 0.95'''
    
    df = copy.copy(df_input)       # работа происходит с копией входной таблицы из-за особеноостей библиотеки pandas
    anomaly_samples_total = []     # список, содержащий номера аномальных образцов
    anomaly_porosity_total = []    # список, содержащий значения пористости аномальных образцов (нужен при визуализации)
    anomaly_TC_total = []          # список, содержащий значения теплопроводности аномальных образцов  
    anomaly_samples = [1]          # т.к. поиск образцов происходит через цикл, изначальная длина должна быть ненулевой
    df.dropna(subset=[porosity, TC], axis=0, inplace=True)   # отбрасываем пустые ячейки 
    
    while len(anomaly_samples) > 0:
        
        anomaly_samples = []
        number_anomaly = []
        lm = LinearRegression()                                      # предполагается зависимость y = l*exp(k*x),
        lm.fit(df[[porosity]], np.log(df[TC]).values.reshape(-1, 1)) # при взятии логарифма имеем ln(y) = ln(l) + k*x
        ln_predicted = lm.predict(df[[porosity]])                    # обозначения: l — теплопроводность матрицы
                                                                     # x — пористость, y — теплопроводность образца;
        value_matrix = m.exp(lm.intercept_)
        k = lm.coef_[0] 

        df['value_trend'] = value_matrix*np.exp(k*df[porosity])     # предсказываем значения по полученному уравнению 
        df['difference'] = df[TC] - df['value_trend']
        sigma = df['difference'].std()
        student_coef = t.interval(confidence_interval, df['difference'].shape[0]-1, loc=0, scale=1)[1] # к-т Стьюдента
        
        anomaly_samples.extend(df[sample][df[abs(df['difference']) > student_coef*sigma].index]) 
        number_anomaly.extend(np.array(df[abs(df['difference']) > student_coef*sigma].index))
        anomaly_samples_total.extend(anomaly_samples)
        
        anomaly_porosity_total.extend(np.array(df[porosity][df[abs(df['difference']) > student_coef*sigma].index]))
        anomaly_TC_total.extend(np.array(df[TC][df[abs(df['difference']) > student_coef*sigma].index]))
        df.drop(index = df[abs(df['difference']) > student_coef*sigma].index, axis=0, inplace = True)
    
    print('СКО регрессии: ', round(sigma, 3), 'Вт/(м*К)')
    
    if draw == True:
        regression_equation = ('y = ' + str('%.3f' % value_matrix) + '*' + 'exp(' + str('%.3f' % k) + '*x)')
        print('Уравнение регрессии:', regression_equation)
        print('Номера аномальных образцов:', anomaly_samples_total)
        plt.figure(figsize=(7,5))
        plt.title('Аномальные образцы')
        plt.xlabel('Пористость, %')
        plt.ylabel('Теплопроводность, TC, Вт/(м*К)')
        plt.scatter(100*df[porosity], df[TC], color='blue')
        plt.scatter(100*np.array(anomaly_porosity_total), anomaly_TC_total, color = 'r')
        x_for_trend = np.linspace(0, max(0.3, df[porosity].max()), 200)
        y_for_trend = value_matrix*np.exp(k*x_for_trend)
        plt.plot(100*x_for_trend, y_for_trend, color='green', linestyle='dashed')
        plt.fill_between(100*x_for_trend, 
                         y_for_trend + student_coef*sigma,
                         y_for_trend - student_coef*sigma, 
                         alpha = 0.2, color = 'orange', label = ('Доверительный интервал, ' + str(confidence_interval)))
        plt.ylim(1,)
        plt.xlim(0, max(30, 100*df[porosity].max()))
        plt.legend()
        plt.grid()
        plt.show()
        
    return copy.copy(df), anomaly_porosity_total, anomaly_TC_total                                        

In [ ]:
def Rel_error(df_input, porosity, TC, TC_fluid, trend):                  # функция построения относительной погрешности
    
    '''Функция предназначена для получения зависимости ошибки теоретического моделирования по формуле Лихтенеккера
       от пористости. 
       Аргументы:
       df_input — входная таблица, тип — Pandas Dataframe;
       porosity — заголовок столбца с пористостью в df_input;
       TC — заголовок столбца с теплопроводностью (как правило, лямбда паралл.) в df_input;
       TC_fluid — теплопроводность флюида (0.025 — для воздуха, 0.60 — для воды, 0.13 — для керосина);
       trend — если значение poly — построение полиномиальной линии тренда второй степени, 
       если linear — лиенйной.'''
    
    df_input.dropna(subset=[porosity, TC], axis=0, inplace=True)  # отбрасываем пустые ячейки 
    df = copy.copy(df_input)                                          
    lm = LinearRegression()                                           # нахождение теплопроводности матрицы аналогично
    lm.fit(df[[porosity]], np.log(df[TC]).values.reshape(-1, 1))      # алгоритму функции Anomaly_exp
    ln_predicted = lm.predict(df[[porosity]])
    TC_matrix = m.exp(lm.intercept_)
    k = lm.coef_[0]
    
    df['y_lich_por'] = (TC_matrix**(1-df[porosity]))*TC_fluid**(df[porosity]) # теплопроводность по формуле Лихтенеккера
    df['rel_error'] = 100*(df['y_lich_por'] - df[TC])/df[TC]                  # относительная погрешность маоделирования
    plt.figure(figsize=(10,5))
    plt.title('''Относительная погрешность теоретического моделирования при заданной пористости и 
              теплопроводности минеральной матрицы''')
    plt.xlabel('Пористость, %')
    plt.ylabel('Относительная погрешность моделирования, %')
    
    mean_rel_err = df['rel_error'][df[porosity] >= 0.1][df[porosity] <= 0.3].mean() # средняя ошибка при пористости 10-30%
    print('Среднее значение относительной погрешности в диапазоне пористости 10-30 % (по образцам):',round(mean_rel_err, 3))
    
    plt.scatter(100*df[porosity], df['rel_error'])
    plt.ylim(min(-50, min(df['rel_error'])),max(0, df['rel_error'].max()))
    plt.xlim(0, max(31, 100*df[porosity].max()))
    plt.vlines(10, min(-50, min(df['rel_error'])), max(0, df['rel_error'].max()), color='orange', linestyles='dashed')
    plt.vlines(30, min(-50, min(df['rel_error'])),max(0, df['rel_error'].max()), color='orange', linestyles='dashed')
    plt.grid()
    
    if trend == 'poly':                                       # добавление линии тренда
        polynomial_features = PolynomialFeatures(degree=2)
        x_poly = polynomial_features.fit_transform(100*df[[porosity]])
        poly_model = LinearRegression()
        poly_model.fit(x_poly, df['rel_error'].values.reshape(-1, 1))
        err_predicted = poly_model.predict(polynomial_features.fit_transform(np.array([10, 30]).reshape(-1,1)))
        print('Диапазон погрешностей при пористости в 10-30% согласно уравнению регресии: [',
              round(err_predicted[1][0], 3), ';', round(err_predicted[0][0], 3), ']', sep='')
        x_for_trend_re = np.linspace(0, max(35, 100*df[porosity].max()), 200)
        err_predicted_full = poly_model.predict(polynomial_features.fit_transform(x_for_trend_re.reshape(-1,1)))
        equation = (str(round(poly_model.coef_[0][-1], 3)) + '*x^2 +(' +
                    str(round(poly_model.coef_[0][-2], 3)) + ')*x + (' + str(round(poly_model.intercept_[0], 3)) + ')')
        print('Уравнение регрессии:', equation)
        poly_feat = polynomial_features.fit_transform(100*df[[porosity]])
        print('Коэффициент детерминации:', r2_score(df['rel_error'], poly_model.predict(poly_feat)))
        plt.plot(x_for_trend_re, err_predicted_full)
    
    elif trend == 'linear':
        lin_mod = LinearRegression()
        lin_mod.fit(100*df[[porosity]], df['rel_error'].values.reshape(-1, 1))
        err_predicted = lin_mod.predict(np.array([10, 30]).reshape(-1,1))
        print('Диапазон погрешностей при пористости в 10-30% согласно уравнению регресии: [',
              round(err_predicted[1][0], 3), ';', round(err_predicted[0][0], 3), ']', sep='')
        x_for_trend_re = np.linspace(0, max(35, 100*df[porosity].max()), 200)
        err_predicted_full = lin_mod.predict(x_for_trend_re.reshape(-1,1))
        print('Уравнение регрессии: y = ',str('%.3f' % lin_mod.coef_[0]),'*x + ', str('%.3f' % lin_mod.intercept_), sep='')
        print('Коэффициент детерминации:', r2_score(df[['rel_error']], lin_mod.predict(100*df[[porosity]])))
        plt.plot(x_for_trend_re, err_predicted_full)

In [ ]:
def Final_graph(df_input, porosity, TC, TC_fluid, TC_matrix, asaad, draw_lich):
    
    '''Функция предназначена для получения визуализации расчетов. Отображаются исходные данные без аномальных образцов,
       линия тренда для модели Лихтенеккера (жёлтая) и Лихтенеккера-Асаада (красная)
       Аргументы:
       df_input — входная таблица, тип — Pandas Dataframe;
       porosity — заголовок столбца с пористостью в df_input;
       TC — заголовок столбца с теплопроводностью (как правило, лямбда паралл.) в df_input;
       TC_fluid — теплопроводность флюида (0.025 — для воздуха, 0.60 — для воды, 0.13 — для керосина);
       asaad — значение коэффициента Асаада;
       draw_lich — если True, на итоговый график добавляется предсказания по модели Лихтенеккера'''
    
    df = copy.copy(df_input)
    plt.figure(figsize=(10,5))
    x_for_trend = np.linspace(0, max(0.3, df[porosity].max()), 200)
    if draw_lich == True:
        y_Lich = (TC_matrix**(1-x_for_trend)) * TC_fluid**(x_for_trend)
        plt.plot(100*x_for_trend, y_Lich, color = 'y', label = 'модель Лихтенеккера')
        
    y_Lich_Asaad = (TC_matrix**(1-asaad*x_for_trend))*TC_fluid**(asaad*x_for_trend)
    
    plt.scatter(100*df[porosity], df[TC], label='Результаты эксперимента')
    plt.plot(100*x_for_trend, y_Lich_Asaad, color='r', label=('модель Лихтенеккера-Асаада, f = ' + str(round(asaad, 3))))
    plt.xlabel("Пористость, %", fontsize=15)
    plt.ylabel("Теплопроводность, Вт/(м*К)", fontsize=15)
    plt.title('Теплопроводность vs пористость', fontsize=17)
    plt.ylim(1,)
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
def Asaad_coefficient(df_in, porosity, TC, TC_fluid, sample, draw=True, trend='linear', title='', confidence_interval=0.95):
    
    '''Главная функция, предназначена для нахождения коэффициента Асаада и визуализации всех данных.
       Аргументы:
       title — Название месторождения. Нужна для удобства, прописывать необязательно;
       df_in — входная таблица, тип — Pandas Dataframe;
       porosity — заголовок столбца с пористостью в df_input;
       TC — заголовок столбца с теплопроводностью (как правило, лямбда паралл.) в df_input;
       TC_fluid — теплопроводность флюида (0.025 — для воздуха, 0.60 — для воды, 0.135 — для керосина);
       sample — заголовок столбца с названиями/номерами образцов, нужен для функции Anomaly_exp;
       draw — если значение True — визуализация исходных данных и выделение аномальных образцов (функция Anomaly_exp);
       trend — если значение True — построение полиномиальной линии тренда второй степени (функция Rel_error)'''
    
    print(title)
    df  = Anomaly_exp(df_in, porosity=porosity, TC=TC, draw=draw, sample=sample,
                      confidence_interval=confidence_interval)[0]  # убираем аномальные образцы
    Rel_error(df, porosity=porosity, TC=TC, TC_fluid=TC_fluid, trend=trend)
    
    lm = LinearRegression()
    lm.fit(df[[porosity]], np.log(df[TC]).values.reshape(-1, 1))
    ln_predicted = lm.predict(df[[porosity]])
    TC_matrix = m.exp(lm.intercept_)
    k = lm.coef_[0]
    
    # поиск коэффициента Асаада. Коэффициент находится перебором 
    
    asaad_coeff = 0
    df['TC теор. Л-А'] = (TC_fluid**(asaad_coeff*df[porosity]))*(TC_matrix**(1-asaad_coeff*df[porosity]))
    
    mse_history = []
    asaad_coeff_array = np.linspace(0, 1.5, 5000)

    for i in range(len(asaad_coeff_array)):
        df['TC теор. Л-А'] = ((TC_fluid**(asaad_coeff_array[i] * df[porosity]))
                              * (TC_matrix**(1-asaad_coeff_array[i]*df[porosity])))
        mse_history.append(mean_squared_error(df[TC], df['TC теор. Л-А']))
    
    # график к-т - Среднеквадратичная ошибка. Для визуализации надо убрать решётки 
    
  #  plt.figure(figsize=(10,7))
  #  plt.plot(asaad_coeff_array, mse_history)
  #  plt.xlabel('Коэффициент Асаада', fontsize=15)
  #  plt.ylabel('Среднеквадратичная ошибка', fontsize=15)
  #  plt.grid()
  #  plt.show()
    
    asaad_coeff = asaad_coeff_array[np.argmin(mse_history)]
    asaad_coeff = round(asaad_coeff, 3)
    
    df['TC теор. Л-А'] = (TC_fluid**(asaad_coeff*df[porosity]))*(TC_matrix**(1-asaad_coeff*df[porosity]))
    
    x_for_trend = np.linspace(0, max(0.3, df[porosity].max()), 200)
    y_for_trend = TC_matrix*np.exp(k*x_for_trend)
    y_Lich_Asaad = (TC_matrix**(1-asaad_coeff*x_for_trend))*TC_fluid**(asaad_coeff*x_for_trend)
    r_squared_2 = r2_score(df[TC], df['TC теор. Л-А'])
    
    # сравнение результатов
    plt.show()
    plt.figure(figsize=(10,5))
    plt.title('''Сравнение экспериментальных данных о теплопроводности образцов и результатов теоретической модели 
                Лихтенеккера-Асаада ''')
    plt.xlabel('Теплопроводность экспериментальная, Вт/(м∙К)', fontsize=15)
    plt.ylabel('Теплопроводность модел., Вт/(м∙К)', fontsize=15)
    
    lm_models = LinearRegression()
    lm_models.fit(df[[TC]], df['TC теор. Л-А'].values.reshape(-1, 1))
    df['TC predicted comparison'] = lm_models.predict(df[[TC]])
    r_models = r2_score(df[[TC]], df['TC predicted comparison'])
    print('Уравнение регрессии: y =', str('%.3f' % lm_models.coef_[0]),'*x +', str('%.3f' % lm_models.intercept_))
    print('Коэффициент детерминации:', r_models)
    plt.scatter(df[[TC]], df['TC теор. Л-А'])
    x_for_trend_model = np.linspace(df[[TC]].min(), df[[TC]].max(), 200)
    y_for_trend_model = lm_models.coef_[0]*x_for_trend_model + lm_models.intercept_
    
    plt.xlim(0.95*min(df[TC].min(), min(y_for_trend_model)), 1.05*max(df[TC].max(), y_for_trend_model.max()))
    plt.ylim(0.95*min(df[TC].min(), min(y_for_trend_model)), 1.05*max(df[TC].max(), y_for_trend_model.max()))
    bisectrix = np.linspace(min(df[TC].min(), min(y_for_trend_model)), 
                           max(df[TC].max(), y_for_trend_model.max()), 200)
    
    plt.plot(bisectrix, bisectrix, color='orange', linestyle='dashed',
             label ='Биссектриса, соответствующая равенству теплопроводностей')
    plt.grid()
    plt.plot(x_for_trend_model, y_for_trend_model, color='r', label='Линия тренда')
    plt.legend()
    plt.show()
    print('Коэффициент Асаада:', asaad_coeff)
    print('Коэффициент Асаада:', round(asaad_coeff, 3))
    r_squared_2 = r2_score(df[TC], df['TC теор. Л-А'])
    print('Коэффициент детерминации теор. модели:', round(r_squared_2,3))
    
    Final_graph(df, porosity, TC, TC_fluid, TC_matrix, asaad_coeff, draw_lich=True)
    
    # Разность между экспериментальными и теоретическими значениями теплопроводностей
    plt.figure(figsize=(10,5))
    print('Диапазон абсолютных погрешностей, Вт/(м*К): ', '[', round((df['TC теор. Л-А']-df[TC]).min(), 3), ', ',
          round((df['TC теор. Л-А']-df[TC]).max(), 3), ']', sep='')
    plt.scatter(100*df[[porosity]], (df['TC теор. Л-А']-df[TC]))
    plt.hlines(0, 0, max(30, 100*df[porosity].max()), linestyles='solid', colors='y')
    if TC_fluid == 0.025:
        word = 'высушенных образцов'
    elif TC_fluid == 0.60:
        word = 'водонасыщенных образцов'
    else:
        word = 'образцов, насыщенных керосином'
    plt.title('Разность между экспериментальными и теоретическими значениями теплопроводностей \n для ' + word +
              ', коэффициент Асаада = ' + str(round(asaad_coeff, 3)))
    plt.xlabel('Пористость, %', fontsize=15)
    plt.ylabel('Теплопроводность, TC, Вт/(м∙К)', fontsize=15)
    plt.xlim(0, max(30, 100*df[porosity].max()))
    plt.grid()
    plt.show()

    # То же, но относительная ошибка
    plt.figure(figsize=(10,5))
    print('Диапазон относительных погрешностей, %: ', '[', round((100*(df['TC теор. Л-А']-df[TC])/df[TC]).min(), 3), ', ',
          round((100*(df['TC теор. Л-А']-df[TC])/df[TC]).max(), 3), ']', sep='')
    plt.scatter(100*df[[porosity]], 100*(df['TC теор. Л-А']-df[TC])/df[TC])
    plt.hlines(0, 0, max(30, 100*df[porosity].max()), linestyles='solid', colors='y')
    if TC_fluid == 0.025:
        word = 'высушенных образцов'
    elif TC_fluid == 0.60:
        word = 'водонасыщенных образцов'
    else:
        word = 'образцов, насыщенных керосином'
    plt.title('Относительная погрешность моделирования \n для ' + word +
              ', коэффициент Асаада = ' + str(round(asaad_coeff, 3)))
    plt.xlabel('Пористость, %', fontsize=12)
    plt.ylabel('ОТносительная погрешность \n моделирования, %', fontsize=12)
    plt.xlim(0, max(30, 100*df[porosity].max()))
    plt.grid()
    plt.show()

## Загрузка данных
Нужно прописывать директорию на своём компьютере или путь на Googe Drive 

In [ ]:
df_tp_1 = pd.read_excel(r'C:\Users\Захар\Desktop\Skoltech\УЧЕБНАЯ ЛИТЕРАТУРА\Ю.А.Попов\База с теплопроводностью и вычислениями\Данные КТС.xlsx',
                  sheet_name = 'Тимано-Печорская-1',
                     skiprows=2)

df_tp_2 = pd.read_excel(r'C:\Users\Захар\Desktop\Skoltech\УЧЕБНАЯ ЛИТЕРАТУРА\Ю.А.Попов\База с теплопроводностью и вычислениями\Данные КТС.xlsx',
                  sheet_name = 'Тимано-Печорская-2',
                     skiprows=2)

df_vs = pd.read_excel(r'C:\Users\Захар\Desktop\Skoltech\УЧЕБНАЯ ЛИТЕРАТУРА\Ю.А.Попов\База с теплопроводностью и вычислениями\Данные КТС.xlsx',
                  sheet_name = 'Восточная Сибирь',
                     skiprows=2)

df2_lithotype_6 = copy.copy(df_tp_2[df_tp_2['Lithotype']==6.0])  # df_tp_2 состоит из двух литотипов, поэтому
df2_lithotype_7 = copy.copy(df_tp_2[df_tp_2['Lithotype']==7.0])  # здесь они разделяются на две независимых таблицы

df_tver = pd.read_excel(r'C:\Users\Захар\Desktop\Skoltech\УЧЕБНАЯ ЛИТЕРАТУРА\Ю.А.Попов\Мое\Тверь. 2- РЕЗУЛЬТАТЫ ЭКСПЕРИМЕНТА.xlsx',
                  sheet_name = 'нас._180 г на л', #'сухие образцы', # нас._0,6 г на л
                  skiprows=1)

In [ ]:
df_bsh = pd.read_excel(r'C:\Users\Захар\Desktop\Skoltech\УЧЕБНАЯ ЛИТЕРАТУРА\Ю.А.Попов\База с теплопроводностью и вычислениями\Коллекции\Тепловые свойства -два флюида.xlsx',
                  sheet_name = 'Балтийский щит')

df_ns = pd.read_excel(r'C:\Users\Захар\Desktop\Skoltech\УЧЕБНАЯ ЛИТЕРАТУРА\Ю.А.Попов\База с теплопроводностью и вычислениями\Коллекции\Тепловые свойства -два флюида.xlsx',
                  sheet_name = 'Нижневартовский свод')

df_cs = pd.read_excel(r'C:\Users\Захар\Desktop\Skoltech\УЧЕБНАЯ ЛИТЕРАТУРА\Ю.А.Попов\База с теплопроводностью и вычислениями\Коллекции\Тепловые свойства -два флюида.xlsx',
                  sheet_name = 'Красноленинский свод')

df_5 = pd.read_excel(r'C:\Users\Захар\Desktop\Skoltech\УЧЕБНАЯ ЛИТЕРАТУРА\Ю.А.Попов\База с теплопроводностью и вычислениями\Коллекции\Тепловые свойства - терригенные породы (два флюида) (1).xlsx',
                  sheet_name = 'Коллекция 5')

#df_1_4 = copy.copy(df_1[df_1['Lithotype']==4])
#df_3_a = copy.copy(df_3[df_3['Mineral type']=='2a'])
df_5_2 = copy.copy(df_5[df_5['Mineral type']=='2b'])

df_ns_1 = copy.copy(df_ns[df_ns['Mineral type']=='1c'])

#df_9_1 = copy.copy(df_9[df_9['Mineral type']=='6a'])

#df_1_4['Thermal conductivity, W/(m*K), Dry, Lpar'] = df_1_4['Thermal conductivity, W/(m*K), Dry, Lpar'].astype(float)

In [ ]:
df_5['Mineral type'].value_counts()

2b    36
10     4
4      2
1b     2
Name: Mineral type, dtype: int64

## Поиск коэффициента Асаада

In [ ]:
Asaad_coefficient(title='Коллекция 1', # в принципе, title можно не прописывать)
                  df_in=df2_lithotype_6,                                  # прописывается в графе "Загрузка данных"
                  porosity='Porosity, Client, EXP',    # заголовок столбца с пористостью из df_in 
                  TC='Lpar, TC, dry, W/(m*K)',  # заголовок столбца с теплопроводностью из df_in
                  TC_fluid=0.025,                                  # 0.025 для воздуха, 0.60 для воды, 0.13 для керосина
                  sample='№ sample',                              # заголовок столбца с наименованиями образцов из df_in
                  draw=True,                                      # True по умолчанию, прописывать не обязательно
                  trend='linear',                                 # linear для линейной зависимости (по умолчанию),
                                                                  # poly для полиномиальной
                  confidence_interval=0.95)                      # 0.95 по умолчанию, прописывать не обязательно

NameError: ignored